In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    train=True,
    root="data",
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
learning_rate = 1e-3
batch_size=64
epochs=5

In [6]:
loss_fn = nn.CrossEntropyLoss()

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
def train_loop(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        #compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred,y)

        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #display progress
        if(batch%100==0):
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [12]:
def test_loop(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    test_loss,correct=0, 0
    with torch.no_grad():
        for X,y in dataloader:
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train_loop(train_dataloader,model,loss_fn,optimizer)
    test_loop(train_dataloader,model,loss_fn)
print("Done!")

Epoch 1
---------------------
loss: 2.310588 [    0/60000]
loss: 2.296625 [ 6400/60000]
loss: 2.277793 [12800/60000]
loss: 2.270941 [19200/60000]
loss: 2.252613 [25600/60000]
loss: 2.244952 [32000/60000]
loss: 2.236376 [38400/60000]
loss: 2.214006 [44800/60000]
loss: 2.213556 [51200/60000]
loss: 2.185674 [57600/60000]
Test Error: 
 Accuracy: 45.9%, Avg loss: 2.175870 

Epoch 2
---------------------
loss: 2.189327 [    0/60000]
loss: 2.177869 [ 6400/60000]
loss: 2.124811 [12800/60000]
loss: 2.137239 [19200/60000]
loss: 2.090018 [25600/60000]
loss: 2.047457 [32000/60000]
loss: 2.065384 [38400/60000]
loss: 1.994158 [44800/60000]
loss: 1.999507 [51200/60000]
loss: 1.936454 [57600/60000]
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.927357 

Epoch 3
---------------------
loss: 1.959764 [    0/60000]
loss: 1.931254 [ 6400/60000]
loss: 1.819263 [12800/60000]
loss: 1.861340 [19200/60000]
loss: 1.753526 [25600/60000]
loss: 1.704714 [32000/60000]
loss: 1.732864 [38400/60000]
loss: 1.630427 [44800/6